In [1]:
import ScraperFC as src
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import VerticalPitch
import seaborn as sns

#este codigo escrapea la pagina de sofascore

#ideal crear set de datos propios para no depender

C:\Users\eriki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
src = src.Sofascore()

In [3]:
src

In [ ]:
data = src.scrape_league_stats('23/24','EPL')
data

: 

In [ ]:
data.columns

: 

In [ ]:
data.to_csv()

: 

In [ ]:
df = src.scrape_league_stats('23/24','EPL',accumulation='per90')
df.head()

: 

In [ ]:
grafo = df.groupby('team').expectedGoals.mean().sort_values(ascending=False)
grafo.plot.bar()

: 

In [ ]:
#eliminar al link original el texto "id:" https://www.sofascore.com/es/millonarios-junior-barranquilla/fxcsrxc#id:11913838
link = 'https://www.sofascore.com/es/millonarios-junior-barranquilla/fxcsrxc#11913838'
win = src.match_momentum(link)

: 

In [ ]:
win_local = win[win['value'] > 0]
win_visitante = win[win['value'] < 0]

plt.bar(win_local.minute , win_local.value,color='red')
plt.bar(win_visitante.minute , win_visitante.value,color='blue')

: 

In [ ]:
dato_partido = src.get_general_match_stats(link)
dato_partido



: 

In [ ]:
src.get_team_names(link)

: 

In [ ]:
jugadores = src.get_players_match_stats(link)
jugadores[0]

: 

In [ ]:
prom_position =src.get_players_average_positions(link)

fig, ax = plt.subplots()

pitch = VerticalPitch(pitch_type='opta')
pitch.draw(ax=ax)
pitch.scatter(prom_position[0].averageX, prom_position[0].averageY, ax=ax)


prom_position[0]

: 

In [ ]:
mapa = src.get_player_heatmap(link, player='Stiven Vega')
fig, ax = plt.subplots()

cmap = 'hot'
pitch = VerticalPitch(pitch_type='opta')
pitch.draw(ax=ax)
pitch.kdeplot(mapa.x,mapa.y,ax=ax,
              levels=100,
              fill=True,
              zorder=-1,
              shade_lowest=True,
              cmap='OrRd')


: 

## Link Cleaner, llego aquí!!!

usa el maravilloso link cleaner para obtener datos sin errores en el scraper

In [4]:
#Limpiador de links, Usalo y veras la magia
def link_cleaner(link):
    str = link.replace("id:","")
    print("* ", str)
    return str 


## DataFrames:
El escraper publica los datos en dataframes, la funcion iloc se usa para manejar los datos como si fuera un array, sin embargo puede manejarlo como objeto, ya que es un array de objetos.

In [6]:
#Implementacion ERIKFORA12

'''
ADVERTENCIA!!.
El ID del link es solo codigo de liga # ID_partida.

ejemplo:
  -link original: https://www.sofascore.com/granada-cadiz/NObsEAn#id:11369454
  -NObsEAn  #  11369454 -> sin espacios
  -quedara así: NObsEAn#11369454

- Sí lo pone de otra manera fallará.
'''
#se definio la funcion para facilitar las iteraciones
def scrap_corners(match_link,index, name, arr): 
   import traceback

   #Datos
   link = link_cleaner(match_link)
 
   try:
     data = src.get_general_match_stats(link)
   except:
     traceback.print_exc()  
   #Nombre equipos
   home = src.get_team_names(link)[0]
   visitor = src.get_team_names(link)[1]

   #tiros de esquina
   home_kicks = data.iloc[index].homeValue
   visitor_kicks = data.iloc[index].awayValue
   
   print(home,": ", int(home_kicks))
   print(visitor,": ",int(visitor_kicks))

   if home == name:
      arr.append(home_kicks)
   elif visitor == name:
      arr.append(visitor_kicks)
   else:
     print(repr("Name lost"))
     

In [13]:
#funcion para calcular los partidos
def calcular_esquinas(arr_links,equipo):
    esquinas = []

    for links in arr_links:
        scrap_corners(links, 6, equipo, esquinas)

    suma_esquinas = 0
    print(esquinas)

    for valor in esquinas:
       suma_esquinas += valor

    promedio = suma_esquinas/ len(esquinas)

    print("promedio: ", promedio)
    print("minimo:", min(esquinas))
    print("maxmimo:", max(esquinas))


links_equipo_1 = ["https://www.sofascore.com/independiente-santa-fe-deportes-tolima/lxcsqxc#id:11435080",
                  "https://www.sofascore.com/millonarios-independiente-santa-fe/qxcsrxc#id:11435086",
                  "https://www.sofascore.com/independiente-santa-fe-atletico-huila/excsqxc#id:11748904",
                  "https://www.sofascore.com/once-caldas-independiente-santa-fe/qxcstxc#id:11783345",
                  "https://www.sofascore.com/independiente-santa-fe-envigado-fc/oxcsqxc#id:11920629"
                  ]

calcular_esquinas(links_equipo_1,"Independiente Santa Fe")
   
links_equipo_2 = ["https://www.sofascore.com/deportivo-la-equidad-deportivo-pasto/nxcsAOc#id:11435042",
                  "https://www.sofascore.com/deportivo-la-equidad-deportivo-pereira/jxcsAOc#id:11435070",
                  "https://www.sofascore.com/rionegro-aguilas-doradas-deportivo-la-equidad/AOcsUou#id:11435084",
                  "https://www.sofascore.com/deportivo-la-equidad-atletico-bucaramanga/dxcsAOc#id:11435109",
                  "https://www.sofascore.com/deportivo-la-equidad-millonarios/rxcsAOc#id:11783467"
                  ]

calcular_esquinas(links_equipo_2,"Deportivo La Equidad")

*  https://www.sofascore.com/independiente-santa-fe-deportes-tolima/lxcsqxc#11435080
Deportes Tolima :  3
Independiente Santa Fe :  0
*  https://www.sofascore.com/millonarios-independiente-santa-fe/qxcsrxc#11435086
Independiente Santa Fe :  2
Millonarios :  0
*  https://www.sofascore.com/independiente-santa-fe-atletico-huila/excsqxc#11748904
Atlético Huila :  5
Independiente Santa Fe :  4
*  https://www.sofascore.com/once-caldas-independiente-santa-fe/qxcstxc#11783345
Independiente Santa Fe :  1
Once Caldas :  2
*  https://www.sofascore.com/independiente-santa-fe-envigado-fc/oxcsqxc#11920629
Independiente Santa Fe :  0
Envigado FC :  1
[0, 2, 4, 1, 0]
promedio:  1.4
minimo: 0
maxmimo: 4
*  https://www.sofascore.com/deportivo-la-equidad-deportivo-pasto/nxcsAOc#11435042
Deportivo La Equidad :  3
Deportivo Pasto :  1
*  https://www.sofascore.com/deportivo-la-equidad-deportivo-pereira/jxcsAOc#11435070
Deportivo Pereira :  1
Deportivo La Equidad :  3
*  https://www.sofascore.com/rionegro-ag